## Regression

In [1]:
import pandas as pd
from math import exp
from tqdm import tqdm_notebook
from sklearn.metrics import roc_auc_score

In [2]:
name = 'data-logistic.csv'
data = pd.read_csv(name, header=None)

X = data.loc[:, 1:]
y = data.loc[:, 0]

In [3]:
def upd_weights(X, y, w1, w2, k, C=0):
    summ1 = 0
    summ2 = 0
    l = len(X)
    
    for i in range(l):
        summ1 += y[i]*X.iloc[i,0]*(1 - (1/(1 + exp(-y[i]*(w1*X.iloc[i,0] + w2*X.iloc[i,1])))))
        
        summ2 += y[i]*X.iloc[i,1]*(1 - (1/(1 + exp(-y[i]*(w1*X.iloc[i,0] + w2*X.iloc[i,1])))))
    
    w1 = w1 + (k/float(l))*summ1 - k*float(C)*w1
    w2 = w2 + (k/float(l))*summ2 - k*float(C)*w2
    
    return [w1, w2]

In [4]:
def descent(X, y, w1, w2, k, C=0):
    w1_upd = w1
    w2_upd = w2
    
    for i in tqdm_notebook(range(10**4)):
        w1_upd, w2_upd = upd_weights(X, y, w1, w2, k, C)
        
        if((w1_upd-w1)**2 + (w2_upd-w2)**2)**0.5 < 1e-5:
            break
        w1, w2 = w1_upd, w2_upd
    return [w1, w2]

In [5]:
def a(x,w1, w2):
    return 1/(1 + exp(-w1*x[1] - w2*x[2]))

In [9]:
weights = [0,0]

weights = descent(X, y, weights[0], weights[1], k=1)
weights_upd = descent(X, y, weights[0], weights[1], k=1, C=15)
print('weights: ' + str(weights))
print('weights_upd: ' + str(weights_upd))

OverflowError: math range error

In [ ]:
y_score = X.apply(lambda x: a(x, weights[0], weights[1]), axis=1)
y_score_upd = X.apply(lambda x: a(x, weights_upd[0], weights_upd[1]), axis=1)

auc = roc_auc_score(y, y_score)
auc_upd = roc_auc_score(y, y_score_upd)

print(round(auc, 3), round(auc_upd, 3))